In [1]:
import os
import re
import ast
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

import torch.nn as nn
from torch.optim import AdamW

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 8
BERT_SPECIES = 'roberta-base'
INPUT_CATS = []
OUTPUT_CATS = ['src','rel','trg','mis']
EPOCHS = 5
LEARNING_RATE = 1e-5
LOGGING_INTERVAL = 100

In [4]:
!ls ../input/mismatch-data/syntax-complete_test.csv

ls: cannot access '../input/mismatch-data/syntax-complete_test.csv': No such file or directory


In [5]:
rawdf = {}
for split in ['train','val','test']:
    dpath = f"../../data/complete_{split}.csv"
    rawdf[split] = pd.read_csv(dpath)

In [6]:
rawdf['train']

Unnamed: 0    id                                              claim  \
0           2047  2145            Ederson Moraes plays only as a forward.   
1           1631  1719  A Song of Ice and Fire is by someone who was b...   
2           1026  1079  Beautiful couldn't chart on the Billboard Hot ...   
3           5605  5824  Jacqueline Kennedy Onassis is forgotten for he...   
4            676   712  Nikola Tesla didn't receive an education in ph...   
...          ...   ...                                                ...   
6438        4537  4719                               Inhumans is a movie.   
6439        2888  3022          Indian was solely produced by Mel Gibson.   
6440        5649  5872                    Elle Fanning is not an actress.   
6441        6778  7052   The New Adventures of Old Christine is a statue.   
6442        5699  5926  Chokher Bali was nominated for Best Actor at t...   

                                                context  \
0     Ederson Santana de Moraes -LRB- born 17 August...   
1     A Song of Ice and Fire is a series of epic fan...   
2     The song peaked at number two on the Billboard...   
3     She is remembered for her lifelong contributio...   
4     Born and raised in the Austrian Empire , Tesla...   
...                                                 ...   
6438  Marvel 's Inhumans , or simply Inhumans , is a...   
6439  Indian -LRB- Tamil : à®‡à®¨à¯à®¤à®¿à®¯à®©à¯ ...   
6440  Mary Elle Fanning -LRB- born April 9 , 1998 -R...   
6441  The New Adventures of Old Christine -LRB- ofte...   
6442  Chokher Bali won the National Film Award for B...   

                                   Source    SSpan             Relation  \
0                          Ederson Moraes  [0, 14]           plays only   
1                  A Song of Ice and Fire  [0, 22]                   is   
2                               Beautiful   [0, 9]       couldn't chart   
3              Jacqueline Kennedy Onassis  [0, 26]         is forgotten   
4                            Nikola Tesla  [0, 12]       didn't receive   
...                                   ...      ...                  ...   
6438                             Inhumans   [0, 8]                   is   
6439                               Indian   [0, 6]  was solely produced   
6440                         Elle Fanning  [0, 12]               is not   
6441  The New Adventures of Old Christine  [0, 35]                   is   
6442                         Chokher Bali  [0, 12]        was nominated   

         RSpan                                    Target     TSpan  \
0     [15, 25]                              as a forward  [26, 38]   
1     [23, 25]  by someone who was born Krzysztof Komeda  [26, 66]   
2     [10, 24]                  on the Billboard Hot 100  [25, 49]   
3     [27, 39]                          for her elegance  [40, 56]   
4     [13, 27]                   an education in physics  [28, 51]   
...        ...                                       ...       ...   
6438   [9, 11]                                   a movie  [12, 19]   
6439   [7, 26]                             by Mel Gibson  [27, 40]   
6440  [13, 19]                                an actress  [20, 30]   
6441  [36, 38]                                  a statue  [39, 47]   
6442  [13, 26]    for Best Actor at the Locarno Festival  [27, 65]   

            mislocation                                           Mismatch  \
0           Target-Head                                    as a goalkeeper   
1           Target-Head  by the American novelist and screenwriter Geor...   
2           Target-Head                               peaked at number two   
3         Relation-Head                                      is remembered   
4     Relation-Modifier                                           received   
...                 ...                                                ...   
6438        Target-Head             an upcoming American television series   
6439  

In [7]:
input_cats = INPUT_CATS
output_cats = OUTPUT_CATS
include_context = True

In [8]:
cat2col = {'src':'SSpan','rel':'RSpan','trg':'TSpan','mis':'MSpan'}
col2cat = {col:cat for cat,col in cat2col.items()}

proc_data = {}
for split in ['train','val','test']:
    proc_data[split] = []
    print(split)
    for _, example in tqdm(rawdf[split].iterrows(),total=len(rawdf[split])):
        set_complete = True
        example_feats = {
            'text': None,
            'input_cols': {},
            'output_cols': {},
        }
#         text = example['data'].strip() if include_context else example['data'].strip().split('\n')[0]
        text = example['claim'] + ' [SEP] '
#         text = example['claim'] + ' <SEP> '

        #         print(text)
        for io_type, cats in {
            'input_cols': input_cats,
            'output_cols': output_cats,
        }.items():
            for cat in cats:
                col = cat2col[cat]
                span = ast.literal_eval(example[col])
                if any([v==None for v in span]):
                    set_complete = False
                else:
                    if cat == 'mis': 
                        span[0] += len(text)
                        span[1] += len(text)
                    example_feats[io_type][col] = span
        text += example['context']
        example_feats['text'] = text
        if set_complete:
            proc_data[split].append(example_feats)

train


  0%|          | 0/6443 [00:00<?, ?it/s]

val


  0%|          | 0/806 [00:00<?, ?it/s]

test


  0%|          | 0/806 [00:00<?, ?it/s]

In [9]:
pprint(proc_data['train'][0]) 

{'input_cols': {},
 'output_cols': {'MSpan': [181, 196],
                 'RSpan': [15, 25],
                 'SSpan': [0, 14],
                 'TSpan': [26, 38]},
 'text': 'Ederson Moraes plays only as a forward. [SEP] Ederson Santana de '
         'Moraes -LRB- born 17 August 1993 -RRB- , known simply as Ederson , '
         'is a Brazilian professional footballer who plays as a goalkeeper for '
         'English club Manchester City .'}


In [10]:
proc_data['train'][0]['text'][10:83]

'raes plays only as a forward. [SEP] Ederson Santana de Moraes -LRB- born '


## Method - 1: String Control

In [11]:
cat_tag_map = {
    'SSpan': ['<SRC>','</SRC>'],
    'RSpan': ['<REL>','</REL>'],
    'TSpan': ['<TRG>','</TRG>'],
    'MSpan': ['<MIS>','</MIS>'],
}

ext_data = {}
for split in ['train','val','test']:
    ext_data[split] = []
    print(split)
    for example in tqdm(proc_data[split]):
        new_text = ""
        new_text_split = []
        new_col_spans = {}          # character level
        new_col_spans_split = {}    # word level
        ptr = 0
        col_span_list = []
        for io_type in ['input_cols','output_cols']:
            for cat, span in example[io_type].items():
                col_span_list.append([io_type,cat,span])
        for io_type, cat, span in sorted(col_span_list,key=lambda x: x[2]):
            new_text += example['text'][ptr:span[0]]
            new_text_split.extend([v for v in example['text'][ptr:span[0]].strip().split(' ') if v!=''])
            ptr = span[0]
            
            cat_start, cat_start_split = len(new_text), len(new_text_split)
            
            if io_type == 'input_cols':
                tags = cat_tag_map[cat]
                string = tags[0] + ' ' + example['text'][span[0]:span[1]] + ' ' + tags[1]
                new_text += string
                new_text_split.extend(string.strip().split())
            else:
                string = example['text'][span[0]:span[1]]
                new_text += string
                new_text_split.extend([v for v in string.strip().split() if v!=''])
                
            cat_end, cat_end_split = len(new_text), len(new_text_split)
            
            new_col_spans[cat] = [cat_start,cat_end]
            new_col_spans_split[cat] = [cat_start_split, cat_end_split]
            
            ptr = span[1]

        new_text += example['text'][ptr:]
        new_text_split.extend(example['text'][ptr:].strip().split())
        
        wid2col = ['TXT' for _ in range(len(new_text_split))]
        for col, w_span in new_col_spans_split.items():
            for wid in range(*w_span):
                wid2col[wid] = col
        
        ext_data[split].append({
            'text': new_text,
            'text_split': new_text_split,
            'input_cols': {col:new_col_spans[col] for col in example['input_cols'].keys()},
            'output_cols': {col:new_col_spans[col] for col in example['output_cols'].keys()},
            'input_cols_split': {col:new_col_spans_split[col] for col in example['input_cols'].keys()},
            'output_cols_split': {col:new_col_spans_split[col] for col in example['output_cols'].keys()},
            'wid2col': wid2col,
            
        })
#         print(example['text'])
#         print()
#         print(new_text)
#         print()
#         for col, span in new_output_cols.items():
#             print(col)
#             print(new_text[span[0]:span[1]])
        
#         break
#     break

train


  0%|          | 0/6440 [00:00<?, ?it/s]

val


  0%|          | 0/806 [00:00<?, ?it/s]

test


  0%|          | 0/803 [00:00<?, ?it/s]

In [12]:
ext_data['test'][136]

{'text': "Sam Walton founded Australia and New Zealand. [SEP] Samuel Moore `` Sam '' Walton -LRB- March 29 , 1918 -- April 5 , 1992 -RRB- was an American businessman and entrepreneur best known for founding the retailers Walmart and Sam 's Club .",
 'text_split': ['Sam',
  'Walton',
  'founded',
  'Australia',
  'and',
  'New',
  'Zealand',
  '.',
  '[SEP]',
  'Samuel',
  'Moore',
  '``',
  'Sam',
  "''",
  'Walton',
  '-LRB-',
  'March',
  '29',
  ',',
  '1918',
  '--',
  'April',
  '5',
  ',',
  '1992',
  '-RRB-',
  'was',
  'an',
  'American',
  'businessman',
  'and',
  'entrepreneur',
  'best',
  'known',
  'for',
  'founding',
  'the',
  'retailers',
  'Walmart',
  'and',
  'Sam',
  "'s",
  'Club',
  '.'],
 'input_cols': {},
 'output_cols': {'SSpan': [0, 10],
  'RSpan': [11, 18],
  'TSpan': [19, 44],
  'MSpan': [197, 234]},
 'input_cols_split': {},
 'output_cols_split': {'SSpan': [0, 2],
  'RSpan': [2, 3],
  'TSpan': [3, 7],
  'MSpan': [36, 43]},
 'wid2col': ['SSpan',
  'SSpan',


In [13]:
tokenizer = AutoTokenizer.from_pretrained(BERT_SPECIES, add_prefix_space=True)
bert_model = AutoModel.from_pretrained(BERT_SPECIES)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
print(tokenizer.all_special_tokens)
custom_spl_tokens = [tag for cat, tags in cat_tag_map.items() for tag in tags]
# custom_spl_tokens = ['<SEP>'] + [tag for cat, tags in cat_tag_map.items() for tag in tags]
print(custom_spl_tokens)
num_added_toks = tokenizer.add_tokens(custom_spl_tokens, special_tokens=True)
print(num_added_toks)
bert_model.resize_token_embeddings(len(tokenizer))
print(tokenizer.all_special_tokens)
tokenizer.save_pretrained("custom_tokenizer")

['<s>', '</s>', '<unk>', '<pad>', '<mask>']
['<SRC>', '</SRC>', '<REL>', '</REL>', '<TRG>', '</TRG>', '<MIS>', '</MIS>']
8
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


('custom_tokenizer/tokenizer_config.json',
 'custom_tokenizer/special_tokens_map.json',
 'custom_tokenizer/vocab.json',
 'custom_tokenizer/merges.txt',
 'custom_tokenizer/added_tokens.json',
 'custom_tokenizer/tokenizer.json')

In [15]:
!cat here/added_tokens.json

cat: here/added_tokens.json: No such file or directory


In [16]:
enc = {}
for split in ['train','val','test']:
    enc[split] = tokenizer.batch_encode_plus(
    [eg['text_split'] for eg in ext_data[split]],
    max_length=128,
    add_special_tokens=True,
    is_split_into_words=True,
    padding=True,
    truncation=True,
    return_tensors='pt',
    )

In [17]:
enc['val']['input_ids'].shape

torch.Size([806, 128])

In [18]:
pipe_data = {} # pipeline-ready data
for split in ['train','val','test']:
    pipe_data[split] = []
    print(split)
    for i in tqdm(range(enc[split]['input_ids'].shape[0])):
        input_ids = enc[split]['input_ids'][i]
        attention_mask = enc[split]['attention_mask'][i]
        sample = ext_data[split][i]
        output_cats = list(sample['output_cols_split'].keys())
        labels = {cat:[] for cat in output_cats}
        wid2col = sample['wid2col']
        for tid, wid in enumerate(enc[split].word_ids(batch_index=i)):
            if wid == None:
                for cat in output_cats:
                    labels[cat].append(0.0)
            else:
                for cat in output_cats:
                    if wid2col[wid] == cat:
                        labels[cat].append(1.0)
                    else:
                        labels[cat].append(0.0)
        pipe_data[split].append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        })

train


  0%|          | 0/6440 [00:00<?, ?it/s]

val


  0%|          | 0/806 [00:00<?, ?it/s]

test


  0%|          | 0/803 [00:00<?, ?it/s]

In [19]:
class StringControlDataset(Dataset):
    
    def __init__(self,data):
        self.data = data
        self.output_cats = list(sorted(self.data[0]['labels'].keys()))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        sample = self.data[idx]
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        labels = torch.tensor([sample['labels'][col] for col in self.output_cats])
        return input_ids, attention_mask, labels

In [20]:
strctrl_dataset = {}
for split in ['train','val','test']:
    strctrl_dataset[split] = StringControlDataset(pipe_data[split])

In [21]:
strctrl_dataloader = {}
for split in ['train','val','test']:
    strctrl_dataloader[split] = DataLoader(strctrl_dataset[split],
                                           batch_size=BATCH_SIZE,
#                                            shuffle=True
                                          )

In [22]:
class StringControlModel(nn.Module):
    def __init__(self,bert_model,out_classes=1):
        super().__init__()
        self.bert_model = bert_model
        self.out_classes = out_classes
        
#         for name, param in self.bert_model.named_parameters():
#              if any([name.startswith(f"encoder.layer.{layer_id}") for layer_id in [0,1,2,
#                                                                     3,4,5,6,7,8,9,] ]):
#                 param.requires_grad = True
        
        self.fc = nn.Sequential(
#                             nn.Dropout(),
                            nn.Linear(768,768),
                            nn.ReLU(),
                        )
        
        self.classifier = nn.Sequential(
#                             nn.Dropout(),
                            nn.Linear(768,self.out_classes),
                            nn.Softmax(dim=1),
                        )

    def forward(self,input_ids,attention_mask):
        emb = self.bert_model(input_ids=input_ids,attention_mask=attention_mask).last_hidden_state
        x = self.fc(emb)
        out = self.classifier(x)
        return out

In [23]:
model = StringControlModel(bert_model,out_classes=len(strctrl_dataset['train'].output_cats))
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"trainable params: {trainable_params}")
print(f"    total params: {total_params}")
print(f"                : {round(100*trainable_params/total_params,2)}%")

model = model.to(DEVICE)

trainable params: 125245444
    total params: 125245444
                : 100.0%


In [24]:
epochs = EPOCHS
loss_fnc = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),lr=LEARNING_RATE)
logging_interval = LOGGING_INTERVAL

In [25]:
# Training
for e in range(epochs):
    print(f'running epoch {e+1} of {epochs}')
    model.train()
    batch_loss = []
    epoch_loss = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['train']),total=len(strctrl_dataloader['train'])):
        
        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        labels = torch.permute(labels,(0,2,1))
        
        optimizer.zero_grad()
        loss = loss_fnc(out,labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss.append(loss.item())
        batch_loss.append(loss.item())
        if len(batch_loss) > logging_interval:
            batch_loss.pop(0)
        if (bidx+1) % logging_interval == 0:
            print(f'train loss: {np.mean(batch_loss)}')
    print(f"epoch loss: {np.mean(epoch_loss)}")
                    
#         break
#     break

running epoch 1 of 5


  0%|          | 0/805 [00:00<?, ?it/s]

train loss: 18.050842266082764
train loss: 18.368298263549804
train loss: 17.987572059631347
train loss: 17.470529890060426
train loss: 18.038204345703125
train loss: 17.43769772529602
train loss: 17.300419206619264
train loss: 17.700811624526978
epoch loss: 17.79434063360558
running epoch 2 of 5


  0%|          | 0/805 [00:00<?, ?it/s]

train loss: 17.433644218444826
train loss: 18.234555025100708
train loss: 17.902650032043457
train loss: 17.378875303268433
train loss: 17.982149801254273
train loss: 17.39729058265686
train loss: 17.274610652923585
train loss: 17.67629309654236
epoch loss: 17.660879107884
running epoch 3 of 5


...

train loss: 17.415144567489623
train loss: 18.225769453048706
train loss: 17.884107151031493
train loss: 17.370074920654297
train loss: 17.97395294189453
train loss: 17.388060140609742
train loss: 17.26268928527832
train loss: 17.671178789138793
epoch loss: 17.64979253022567
running epoch 4 of 5


  0%|          | 0/805 [00:00<?, ?it/s]

train loss: 17.405021047592165
train loss: 18.21731342315674
train loss: 17.877299280166625
train loss: 17.363894214630125
train loss: 17.972552604675293
train loss: 17.38258366584778
train loss: 17.25912561416626
train loss: 17.664149913787842
epoch loss: 17.64365677567002
running epoch 5 of 5


  0%|          | 0/805 [00:00<?, ?it/s]

train loss: 17.402000274658203
train loss: 18.213174085617066
train loss: 17.874701499938965
train loss: 17.35982479095459
train loss: 17.96691614151001
train loss: 17.38016695022583
train loss: 17.258259992599488
train loss: 17.657357444763182
epoch loss: 17.640008770012706


In [26]:
# Inference
with torch.no_grad():
    model.eval()
    epoch_loss = []
    auto = []
    gold = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['train']),
                            total=len(strctrl_dataloader['train'])):

        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        auto.append(out)
        labels = torch.permute(labels,(0,2,1))
        gold.append(labels)
        loss = loss_fnc(out,labels)
        epoch_loss.append(loss.item())
    print(f"epoch loss: {np.mean(epoch_loss)}")

  0%|          | 0/805 [00:00<?, ?it/s]

epoch loss: 17.632136430207247


In [27]:
auto[136].shape, gold[136].shape

(torch.Size([8, 128, 4]), torch.Size([8, 128, 4]))

In [28]:
# auto_new = [auto[i][:,:,0].reshape((8,128,1)) for i in range(len(auto))]

In [29]:
best = -1
best_thresh = None
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    for thresh in tqdm(np.linspace(0,0.5,200)):
        IOU = []
        for ba, bg in list(zip(auto_new,gold_new))[:100]:
            for i in range(bg.shape[0]):
                sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
                IOU.append(sum(sa & sg)/(1e-5 + sum(sa | sg)))
        score = torch.mean(torch.tensor(IOU))
        if score > best:
            best = score
            best_thresh = thresh
    print(best, best_thresh)

  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.6383) 0.002512562814070352


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.8860) 0.002512562814070352


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.9950) 0.005025125628140704


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.9950) 0.005025125628140704


In [30]:
# Inference
with torch.no_grad():
    model.eval()
    epoch_loss = []
    auto = []
    gold = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['test']),
                            total=len(strctrl_dataloader['test'])):

        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        auto.append(out)
        labels = torch.permute(labels,(0,2,1))
        gold.append(labels)
        loss = loss_fnc(out,labels)
        epoch_loss.append(loss.item())
    print(f"epoch loss: {np.mean(epoch_loss)}")

  0%|          | 0/101 [00:00<?, ?it/s]

epoch loss: 18.059056461447536


In [31]:
tags = ['MSpan','RSpan', 'SSpan', 'TSpan']
nbname = 'roberta-m3'
for i in range(len(ext_data['test'])):
    ext_data['test'][i]['wid2col_p'] = ext_data['test'][i]['wid2col'][:]
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    cnt = 0
    for ba, bg in tqdm(zip(auto_new,gold_new)):
        for i in range(bg.shape[0]):
            sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
            cnt2 = 0
            for a,b in zip(sa, sg):
                if a == True and b == False:
                    ext_data['test'][cnt]['wid2col_p'][cnt2] = tags[q]
                elif a == False and b == True and ext_data['test'][cnt]['wid2col_p'][cnt2] == tags[q]:
                    # print(cnt, cnt2, q)
                    ext_data['test'][cnt]['wid2col_p'][cnt2] = 'TXT'
                elif a == False and b == True:
                    pass
                    # print(cnt, cnt2, q)
                cnt2 += 1
                if cnt2 == len(ext_data['test'][cnt]['wid2col_p']):
                    break
            cnt += 1

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [32]:
import pickle as pkl
with open('results/' + nbname, 'wb+') as f:
    pkl.dump(ext_data, f)

In [33]:
for i in range(len(auto)):
    try:
        auto[i][:,:,0].reshape((8,128,1)).shape
    except:
        print(i)

100


In [34]:
thresh = float(best_thresh)
IOU = []
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    for ba, bg in zip(auto_new,gold_new):
        for i in range(bg.shape[0]):
            sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
            IOU.append(sum(sa & sg)/(1e-5 + sum(sa | sg)))
    #         break
    #     break
    print(torch.mean(torch.tensor(IOU)))

tensor(0.6170)
tensor(0.7446)
tensor(0.8278)
tensor(0.8355)


In [35]:
# Inference
with torch.no_grad():
    model.eval()
    epoch_loss = []
    auto = []
    gold = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['val']),
                            total=len(strctrl_dataloader['val'])):

        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        auto.append(out)
        labels = torch.permute(labels,(0,2,1))
        gold.append(labels)
        loss = loss_fnc(out,labels)
        epoch_loss.append(loss.item())
    print(f"epoch loss: {np.mean(epoch_loss)}")

  0%|          | 0/101 [00:00<?, ?it/s]

epoch loss: 17.905728056879326


In [36]:
thresh = float(best_thresh)
IOU = []
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    for ba, bg in zip(auto_new,gold_new):
        for i in range(bg.shape[0]):
            sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
            IOU.append(sum(sa & sg)/(1e-5 + sum(sa | sg)))
    #         break
    #     break
    print(torch.mean(torch.tensor(IOU)))

tensor(0.6131)
tensor(0.7523)
tensor(0.8330)
tensor(0.8425)
